In [25]:
from preprocessing import * 
from sklearn import svm
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

from sklearn.multiclass import OneVsRestClassifier

import torch
from torch import nn, optim
from classifier import nn_train, nn_predict, nn_accuracy
from torchvision import datasets, transforms, models
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# convert_data()

# Read and split the data

In [6]:
i = 8

x, y = read_file(i)

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 100, test_size = 0.2)

print("final shapes are: ", X_train.shape, X_test.shape, y_train.shape, y_test.shape)



final shapes are:  (230, 25, 313) (58, 25, 313) (230,) (58,)


In [7]:
train_coeff = featurize(X_train)
coeff_len = len(train_coeff)

csp = [mne.decoding.CSP(8) for _ in range(coeff_len)]
X_train = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x], y_train) for x  in range(coeff_len)),axis=-1)



Computing rank from data with rank=None
    Using tolerance 59 (2.2e-16 eps * 25 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 55 (2.2e-16 eps * 25 dim * 9.9e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 62 (2.2e-16 eps * 25 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 50 (2.2e-16 eps * 25 dim * 9.1e+15  max singular value)
    Estimated rank (

c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 26 (2.2e-16 eps * 25 dim * 4.7e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 24 (2.2e-16 eps * 25 dim * 4.3e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 38 (2.2e-16 eps * 25 dim * 6.8e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 22 (2.2e-16 eps * 25 dim * 4e+15  max singular value)
    Estimated rank (ma

In [8]:


clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
# clf = OneVsRestClassifier(svm.SVC())
# clf = OneVsRestClassifier(clf)
clf.fit(X_train, y_train)

print("Accuracy is ", sum(clf.predict(X_train) == y_train) / len(X_train))
print(len(X_train))

print("test Accuracy is ", sum(clf.predict(X_test) == y_test) / len(X_test))
print(len(X_test))

print("kappa score on train is: ", cohen_kappa_score(clf.predict(X_train), y_train))
print("kappa score on test is: ", cohen_kappa_score(clf.predict(X_test), y_test))


Accuracy is  0.9782608695652174
230
test Accuracy is  0.8275862068965517
58
kappa score on train is:  0.9709705919474947
kappa score on test is:  0.7654670440760211


In [30]:
print(y_test)

[3. 3. 2. 1. 2. 3. 1. 1. 1. 4. 3. 4. 1. 3. 3. 2. 3. 3. 3. 2. 2. 1. 2. 1.
 2. 3. 2. 3. 4. 2. 3. 2. 4. 4. 3. 4. 3. 3. 1. 1. 1. 1. 2. 3. 2. 3. 1. 1.
 3. 3. 4. 4. 1. 1. 1. 4. 3. 1.]


(58, 25, 313) ['F' 'F' 'R ' 'L' 'R ' 'F' 'L' 'L' 'L' 'B' 'F' 'B' 'L' 'F' 'F' 'R ' 'F'
 'F' 'F' 'R ' 'R ' 'L' 'R ' 'L' 'R ' 'F' 'R ' 'F' 'B' 'R ' 'F' 'R ' 'B'
 'B' 'F' 'B' 'F' 'F' 'L' 'L' 'L' 'L' 'R ' 'F' 'R ' 'F' 'L' 'L' 'F' 'F' 'B'
 'B' 'L' 'L' 'L' 'B' 'F' 'L']
